In [ ]:
import sys
import pathlib as pl
import math

import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import matplotlib.pyplot as plt

sys.path.append("../")

import geohexgrid as ghg

%load_ext autoreload
%autoreload 2

DATA_DIR = pl.Path("../tests/data")
NZTM = "epsg:2193"  # New Zealand Transverse Mercator CRS
WGS84 = "epsg:4326"

# Using `make_grid_points` and `make_grid`

Hexagons are named (via the `cell_id` column) by their *double coordinates* as shown here 
and [described in more detail by Red Blob Games](https://www.redblobgames.com/grids/hexagons/#coordinates-doubled) .

In [ ]:
# Make a 3 x 4 hex grid centered at (0, 0) and starting with (southwest) cell -2,-2.
nrows, ncols, R = 3, 4, 1 
a0, b0 = -2, -2  
x0, y0 = ghg.double_to_cartesian(a0, b0, R)
X, Y = ghg.make_grid_points(nrows, ncols, R, x0, y0)
grid = ghg.make_grid(nrows, ncols, R, x0, y0, a0=a0, b0=b0)
display(grid.head())
print(grid["cell_id"].tolist())

# Plot
fig, ax = plt.subplots()
ax.scatter(X, Y, color="black", alpha=0.9)
grid.plot(ax=ax, color="white", edgecolor="red", alpha=0.9, aspect="equal")
# Plot labels taken from https://stackoverflow.com/a/38902492
grid.apply(
    lambda x: ax.annotate(
        text=x['cell_id'], 
        xy=x.geometry.centroid.coords[0], 
        ha='center', 
        color="red"
    ), 
    axis=1
);

# Areas should be correct
assert np.allclose(grid.area, 3 * np.sqrt(3) * R**2 / 2)

In [ ]:
# The grid should be gapless
p = grid.unary_union.boundary
display(p, p.is_ring)

# Here's a non-gapless collection of hexagons
p1 = grid["geometry"].iat[0].buffer(-0.001)
p2 = grid["geometry"].iat[1]
q = p1.union(p2).boundary

display(q, q.is_ring)


# Using `make_grid_from_bounds`

In [ ]:
# Define a rectangle that illustrates an interesting edge case
rect = gpd.GeoDataFrame(geometry=[sg.Polygon([(2.1, -1), (4.9, -1), (4.9, 1.9), (2.1, 1.9)])])
R = 1
grid = ghg.make_grid_from_bounds(*rect.total_bounds, R=R)
display(grid.head())
print(grid["cell_id"].tolist())

base = rect.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

In [ ]:
# Two grids with the same origin should have identical
# cells where they overlap

rect1 = gpd.GeoDataFrame(geometry=[sg.Polygon([(-2, 1), (3, 1), (3, 5), (-2, 5)])])
base = rect1.plot(alpha=0.5)
rect2 = rect1.translate(-1, 1)
rect2.plot(ax=base, color="orange", alpha=0.5)

R = 1
grid1 = ghg.make_grid_from_bounds(*rect1.total_bounds, R=R)
grid2 = ghg.make_grid_from_bounds(*rect2.total_bounds, R=R)
base = grid1.plot(alpha=0.5)
grid2.plot(ax=base, alpha=0.5, color="orange")

cell_ids = set(grid1["cell_id"]) & set(grid2["cell_id"])
g1 = (
    grid1
    .loc[lambda x: x["cell_id"].isin(cell_ids)]
    .sort_values("cell_id", ignore_index=True)
)
g2 = (
    grid2
    .loc[lambda x: x["cell_id"].isin(cell_ids)]
    .sort_values("cell_id", ignore_index=True)
)
g1.geom_equals_exact(g2, tolerance=10e-12).all()


# Using `make_grid_from_gdf`

In [ ]:
shape = gpd.GeoDataFrame(geometry=[sg.Polygon([(1, -1), (3, 1), (0, 3)])])
grid = ghg.make_grid_from_gdf(shape, R=1)
display(grid)

base = shape.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

# With clipping this time
grid2 = ghg.make_grid_from_gdf(shape, R=1, clip=True)
display(grid2)

base2 = shape.plot(color='gray', aspect="equal")
grid2.plot(ax=base2, color="white", edgecolor="red", alpha=0.5)

In [ ]:
shapes = gpd.read_file(DATA_DIR / "shapes.geojson").to_crs(NZTM)
R = 900
grid = ghg.make_grid_from_gdf(shapes, R=R)
display(grid.head())

base = shapes.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

# With clipping this time
base2 = shapes.plot(color='gray', aspect="equal")
grid2 = ghg.make_grid_from_gdf(shapes, R=R, clip=True)
grid2.plot(ax=base2, color="white", edgecolor="red", alpha=0.5)

grid.shape[0] == grid2.shape[0]

In [ ]:
# Cover New Zealand!
nz = gpd.read_file(DATA_DIR / "nz_tas.gpkg")
display(nz.crs)
%time grid = ghg.make_grid_from_gdf(nz, R=10_000) # 10 km circumradius

base = nz.plot(color='black', aspect="equal", figsize=(10, 10))
grid.plot(ax=base, color='white', edgecolor="red", alpha=0.5)

# plt.axis('off')
# plt.savefig('../nz_10000m.png', bbox_inches='tight')

# With clipping this time
%time grid2 = ghg.make_grid_from_gdf(nz, R=10_000, clip=True)

base2 = nz.plot(color='black', aspect="equal", figsize=(10, 10))
grid2.plot(ax=base2, color='white', edgecolor="red", alpha=0.5)


In [ ]:
# # Speed test

# nz = gpd.read_file(DATA_DIR / "nz_tas.gpkg")
# akl = nz.loc[lambda x: x["ta2021_name"] == "Auckland"]
# %time grid = ghg.make_grid_from_gdf(akl, R=250) 

# base = akl.plot(color='black', figsize=(10, 10), aspect="equal")
# grid.plot(ax=base, color='white', edgecolor="red", alpha=0.5)